I wrote two posts 
- [Image_resizing](https://www.kaggle.com/prakashr/the-nature-conservancy-fisheries-monitoring/careful-with-image-resizing)
- [Image Pre-Processing](https://www.kaggle.com/prakashr/the-nature-conservancy-fisheries-monitoring/image-processing-with-opencv)

Now Lets see how to write an Image Generator function to train the neural networks effectively 

In [ ]:
import numpy as np
import pandas as pd 
import glob,cv2
import random
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
from PIL import Image
import os 
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings("ignore")

Lets define a function to return a dictionary with keys as folder names and value as a list of file locations

In [ ]:
def filelist(folder):
    """
    Returns a dictionary with keys as folder names and value as a list of file locations

    Args:
    folder: string of folder name which contain all sub folder of images

    Returns:
    A dictionary object with keys - folder name and values - list of file locations of respective folder
    """
    file_dict={}
    folderlist = glob.glob(folder)
    for i in tqdm(folderlist):
        filelist = glob.glob(i+"/*")
        filename = i.rsplit("/")[-1]
        file_dict[filename]= filelist

    return file_dict

In [ ]:
folderlist = filelist("../input/train/*")

In [ ]:
folderlist["ALB"][0] # sample output

In [ ]:
def translate_xy(image,x,y):
    """
    Return a translated Image of the input Image
    Args:
    image : Input Image you want to translate (shape height * width * channel)
    x,y : -ve x _ left, +ve x _ right, -ve y _ up , +ve y down

    Return :
    Translated Image

    """
    M = np.float32([[1, 0, x],[0, 1, y]])
    shifted = cv2.warpAffine(image,M,(image.shape[1],image.shape[0]))
    return shifted


# rotate
def rotate(image, angle, center=None, scale=1.0):
	"""
	Rotates the Image by a specified angle
	Args:
	image  - an numpy array of the image
	angle  - Angle of rotation
	center - Default None and when used will use Image center for rotation
	scale  - to scale down the image if required (0-1)
	Returns:
	returns an numpy array of rotated Images
	"""
	(h,w) = image.shape[:2]
	if center is None:
		center =(w/2,h/2)
		M = cv2.getRotationMatrix2D(center,angle,scale)
		rotated = cv2.warpAffine(image,M,(w,h))
	return rotated

- Our Image data generator should do the following things.
1) resize the Images to the required size,
2) do transformation as required - Here we randomly rotate, translate and flip each image
3) Scale the Image
4) Yield Images in batches 

In [ ]:
def Dev_Image_data_generator(folderlist,resize = (920,1200),Transformation = True, scaling = True, batch_size = 16):
    """
    Yields a tuple (x,y) with x - batch of images(numpy array), y - image label (numpy array)

    Args:
    folderlist : A dictionary object
    resize : tuple of (x,y)
    Transformation : If True Data Aguementation is done
    scaling : If True , every Image is scaled
    batch_size : The batch_size to yield for every iteration

    returns:
    A tuple with Images and labels as numpy arrays.
    """

    while True:
        total_classes = len(folderlist.keys()) # Number of classes
        keys = folderlist.keys() # string of classes 
        Images = [] 
        Image_label = []
        for key in folderlist.keys():
            img_label = random.choice(folderlist[key]) # select an Image location from each class
            img = Image.open(img_label,'r') # read the Image
            # Resize the Image and paste it on a background (black mat/white mat) of size - resize
            h = resize[1] 
            l = int(img.size[1]*h/img.size[0])
            img = img.resize((h,l), Image.ANTIALIAS)
            background = Image.new('RGB', (resize[1], resize[0]), (0, 0, 0))
            img_w, img_h = img.size
            bg_w, bg_h = background.size
            offset = (int((bg_w - img_w) / 2), int((bg_h - img_h) / 2))
            background.paste(img, offset)
            # convert the Image into numpy array 
            background = np.asarray(background)
            if Transformation == True:
                # rotate, translate and flip the Image randomly
                rotation = rotate(background,random.choice(range(360)))
                translate = translate_xy(background,random.choice(range(int(resize[0]/4))),random.choice(range(int(resize[1]/4))))
                flip = cv2.flip(rotation,1)
                Y = np.concatenate((rotation[np.newaxis,:,:,:],flip[np.newaxis,:,:,:],translate[np.newaxis,:,:,:]))
                Images.append(Y)
                Images.append(background[np.newaxis,:,:,:])
                Image_label.append([key for i in range(4)]) # 4 because we have 4 Images
            else:
                Images.append(background[np.newaxis,:,:,:])
                Image_label.append([key])
        # concatenate all the Images- this will convert list of Images to shape(noofimages,height,width,channels)
        Image_label = np.concatenate(Image_label)
        Images = np.concatenate(Images)
        # Convert Image labels to dummy
        Image_label = np.array(pd.get_dummies(Image_label))
        X_Image , Y_Image = shuffle(Images,Image_label,random_state=0)
        # if scaling is true, divide it by 255.0
        if scaling == True:
            X_Image = X_Image/255.0
        else:
            X_Image = X_Image
        batches = int(len(X_Image)/batch_size)
        # yield a batch of Images 
        for batch in range(batches):
            x = X_Image[batch*batch_size:(batch+1)*batch_size,:,:,:]
            y = Y_Image[batch*batch_size:(batch+1)*batch_size]
            yield((x,y))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

for i in range(5):
    x,y = next(Dev_Image_data_generator(folderlist,resize = (256,334),Transformation = True, scaling = True, batch_size = 4))
    fig,ax = plt.subplots(nrows=1,ncols=4,sharex="col",sharey="row",figsize=(20,3))
    for j,img in enumerate(x):
        ax[j].imshow(img)

This should work fine , In the next post lets see how to implement CNN's using TensorFlo. 